In [1]:
import pandas as pd
import time
from selenium import webdriver

pd.set_option('display.max_rows', 100)

In [2]:
data = pd.DataFrame(data=[], columns=['앱이름','아이디','리뷰','별점','날짜'])

In [3]:
driver = webdriver.Chrome("chromedriver.exe")
url = 'https://play.google.com/store/apps/details?id=com.chucklefish.stardewvalley&hl=ko&gl=US'
driver.get(url)

In [4]:
driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[6]/div/span/span').click()

In [5]:
# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 999999999999)")
    time.sleep(1)

In [74]:
scroll_down(driver)

In [7]:
driver.get('https://play.google.com/store/apps/details?id=com.chucklefish.stardewvalley&hl=ko&gl=US&showAllReviews=true')

In [8]:
# 어플 이름 수집
app_name = driver.find_element_by_css_selector('.AHFaub')
app_name.text

'스타듀 밸리 Stardew Valley'

In [9]:
# 유저 이름 수집
user_names = driver.find_elements_by_css_selector('.X43Kjb')
user_names[0].text

'연치_YeonCh'

In [10]:
# 리뷰 수집
reviews = driver.find_elements_by_css_selector('.UD7Dzf')
reviews[0].text

'진짜 너무 좋은데 폰 멀티 좀ㅠㅠㅠ 대체 언제 업데이트가 되는건지 너무 안타까워요ㅠㅠㅠㅠ 멀티 기능 만들기가 어려운건 당연히 알지만 이 게임 구매한지 2년이 다 되어 가는데 아직도 멀티가 안 나온건 좀... 아쉽네요...😢 멀티 플레이 기능 좀...😭😭 그래도 시간 때우기 정말 좋은 게임이고 재밌습니다! 좋은 게임 감사해요!'

In [78]:
# 별점 수집
star_grades = driver.find_elements_by_xpath('//div[@class="pf5lIe"]/div[@role="img"]')
star_grades[0].get_attribute("aria-label")

'별표 5개 만점에 4.8개를 받았습니다.'

In [79]:
# 날짜 수집
dates = driver.find_elements_by_css_selector('.p2TkOb')
dates[0].text

'2021년 1월 12일'

In [15]:
def crawl(driver, data, k):
    
    app_name = driver.find_element_by_css_selector('.AHFaub')
    user_names = driver.find_elements_by_css_selector('.X43Kjb')
    reviews = driver.find_elements_by_css_selector('.UD7Dzf')
    star_grades = driver.find_elements_by_xpath('//div[@class="pf5lIe"]/div[@role="img"]')
    dates = driver.find_elements_by_css_selector('.p2TkOb')
    
    for i in range(k):
        tmp = []
        tmp.append(app_name.text)
        tmp.append(user_names[i].text)
        tmp.append(reviews[i].text)
        tmp.append(star_grades[i].get_attribute('aria-label'))
        tmp.append(dates[i].text)
        
        tmp = pd.DataFrame(data=[tmp], columns=data.columns)
        data = pd.concat([data,tmp])
        
    print(app_name.text + "앱 리뷰 수집 완료")
    
    return data

In [20]:
print(len(user_names))
print(len(reviews))

40
40


In [18]:
# 스크롤 다운을 한 뒤, 크롤링을 해야 인덱스 에러 안남
scroll_down(driver)

data = crawl(driver,data, 200)

스타듀 밸리 Stardew Valley앱 리뷰 수집 완료


In [21]:
# 인덱스 번호를 다시 0부터 리셋합니다.
data.reset_index(inplace=True, drop=True)

In [22]:
data.head()

,앱이름,아이디,리뷰,별점,날짜
0,스타듀 밸리 Stardew Valley,연치_YeonCh,진짜 너무 좋은데 폰 멀티 좀ㅠㅠㅠ 대체 언제 업데이트가 되는건지 너무 안타까워요ㅠ...,별표 5개 만점에 4.8개를 받았습니다.,2021년 1월 12일
1,스타듀 밸리 Stardew Valley,김세은,진짜 너무너무 재밌고 좋은데 멀티가 안되는게 너무 아쉽다. (진짜 멀티있으면...짱...,별표 5개 만점에 5개를 받았습니다.,2021년 1월 11일
2,스타듀 밸리 Stardew Valley,안소연,게임에 돈 쓴 건 처음이에요 게임 딱 시작할 땐 뭐 어쩌라는 거야 싶었는데 블로그 ...,별표 5개 만점에 5개를 받았습니다.,2021년 1월 12일
3,스타듀 밸리 Stardew Valley,쿠시로,돈을 쓰면서 까지 사고싶었던 게임은 이 게임이 처음입니다. 사서 플레이 하고있는데 ...,별표 5개 만점에 5개를 받았습니다.,2021년 1월 12일
4,스타듀 밸리 Stardew Valley,톰과주예리,조작법이 불편하다는 리뷰가 있던데 참ㅋㅋ 설정에 들어가보신적이 없으신건지 조작법만 ...,별표 5개 만점에 5개를 받았습니다.,2021년 1월 8일


In [23]:
# 별점중 1개~5개만 추출
tmp = data.copy()

In [24]:
# 앞의 별표 5점은 생략
tmp['별점'] = tmp['별점'].apply(lambda x: x[5:])

In [25]:
tmp.head(3)

,앱이름,아이디,리뷰,별점,날짜
0,스타듀 밸리 Stardew Valley,연치_YeonCh,진짜 너무 좋은데 폰 멀티 좀ㅠㅠㅠ 대체 언제 업데이트가 되는건지 너무 안타까워요ㅠ...,만점에 4.8개를 받았습니다.,2021년 1월 12일
1,스타듀 밸리 Stardew Valley,김세은,진짜 너무너무 재밌고 좋은데 멀티가 안되는게 너무 아쉽다. (진짜 멀티있으면...짱...,만점에 5개를 받았습니다.,2021년 1월 11일
2,스타듀 밸리 Stardew Valley,안소연,게임에 돈 쓴 건 처음이에요 게임 딱 시작할 땐 뭐 어쩌라는 거야 싶었는데 블로그 ...,만점에 5개를 받았습니다.,2021년 1월 12일


In [26]:
import re

m = re.compile('[0-9][\.0-9]*')

tmp['별점'] = tmp['별점'].apply(lambda x : m.findall(x)[0])

In [27]:
tmp.head()

,앱이름,아이디,리뷰,별점,날짜
0,스타듀 밸리 Stardew Valley,연치_YeonCh,진짜 너무 좋은데 폰 멀티 좀ㅠㅠㅠ 대체 언제 업데이트가 되는건지 너무 안타까워요ㅠ...,4.8,2021년 1월 12일
1,스타듀 밸리 Stardew Valley,김세은,진짜 너무너무 재밌고 좋은데 멀티가 안되는게 너무 아쉽다. (진짜 멀티있으면...짱...,5,2021년 1월 11일
2,스타듀 밸리 Stardew Valley,안소연,게임에 돈 쓴 건 처음이에요 게임 딱 시작할 땐 뭐 어쩌라는 거야 싶었는데 블로그 ...,5,2021년 1월 12일
3,스타듀 밸리 Stardew Valley,쿠시로,돈을 쓰면서 까지 사고싶었던 게임은 이 게임이 처음입니다. 사서 플레이 하고있는데 ...,5,2021년 1월 12일
4,스타듀 밸리 Stardew Valley,톰과주예리,조작법이 불편하다는 리뷰가 있던데 참ㅋㅋ 설정에 들어가보신적이 없으신건지 조작법만 ...,5,2021년 1월 8일


In [28]:
tmp.to_csv('스타듀밸리_리뷰평점.csv', encoding='utf-8')
tmp.to_excel('스타듀밸리_리뷰평점.xlsx')

In [29]:
re = pd.read_csv('스타듀밸리_리뷰평점.csv', encoding='utf-8')
re.head()

,Unnamed: 0,앱이름,아이디,리뷰,별점,날짜
0,0,스타듀 밸리 Stardew Valley,연치_YeonCh,진짜 너무 좋은데 폰 멀티 좀ㅠㅠㅠ 대체 언제 업데이트가 되는건지 너무 안타까워요ㅠ...,4.8,2021년 1월 12일
1,1,스타듀 밸리 Stardew Valley,김세은,진짜 너무너무 재밌고 좋은데 멀티가 안되는게 너무 아쉽다. (진짜 멀티있으면...짱...,5.0,2021년 1월 11일
2,2,스타듀 밸리 Stardew Valley,안소연,게임에 돈 쓴 건 처음이에요 게임 딱 시작할 땐 뭐 어쩌라는 거야 싶었는데 블로그 ...,5.0,2021년 1월 12일
3,3,스타듀 밸리 Stardew Valley,쿠시로,돈을 쓰면서 까지 사고싶었던 게임은 이 게임이 처음입니다. 사서 플레이 하고있는데 ...,5.0,2021년 1월 12일
4,4,스타듀 밸리 Stardew Valley,톰과주예리,조작법이 불편하다는 리뷰가 있던데 참ㅋㅋ 설정에 들어가보신적이 없으신건지 조작법만 ...,5.0,2021년 1월 8일
